In [ ]:
import openml
import pandas as pd
import json

def get_top_5_sklearn_with_params_cc18():
    print("1. Pobieranie konfiguracji OpenML-CC18...")
    suite = openml.study.get_suite('OpenML-CC18')
    tasks_df = openml.tasks.list_tasks(task_id=suite.tasks, output_format="dataframe")

    binary_tasks = tasks_df[tasks_df['NumberOfClasses'] == 2]
    
    print(f"2. Znaleziono {len(binary_tasks)} zadań klasyfikacji binarnej.")
    print("3. Pobieranie modeli i ich hiperparametrów (może to potrwać)...")

    results = []

    for task_id, task_row in binary_tasks.iterrows():
        print(f"\n--- Zadanie: {task_row['name']} (ID: {task_id}) ---")
        try:
            evals = openml.evaluations.list_evaluations(
                function='area_under_roc_curve', 
                tasks=[task_id], 
                size=20000, 
                output_format='dataframe'
            )

            if evals.empty:
                print("   Brak")
                continue

            evals = evals.sort_values(by='value', ascending=False)
            
            count = 0
            
            for _, run_row in evals.iterrows():
                flow_name = str(run_row['flow_name'])
                
                # Szukamy tylko modeli ze 'sklearn' w nazwie
                if 'sklearn' in flow_name:
                    run_id = run_row['run_id']
                    
                    try:
                        # 3. KLUCZOWE: Pobieramy obiekt RUN, żeby dostać hiperparametry
                        run = openml.runs.get_run(run_id)

                        params_dict = {}
                        for param in run.parameter_settings:
                            key = param['oml:name']
                            value = param['oml:value']
                            params_dict[key] = value
                        
                        params_json = json.dumps(params_dict, indent=None)

                        count += 1
                        results.append({
                            'Task ID': task_id,
                            'Dataset Name': task_row['name'],
                            'Rank': count,
                            'Score (AUC)': run_row['value'],
                            'Flow Name': flow_name, 
                            'Hyperparameters': params_json, 
                            'Run ID': run_id
                        })
                        
                        
                    except Exception as e:
                        print(f"   Błądrun_id {run_id}: {e}")
                    
                    if count >= 5:
                        break
            
            if count == 0:
                print("   Nie znaleziono żadnych modeli sklearn.")

        except Exception as e:
            print(f"   błąd {task_id}: {e}")

    return pd.DataFrame(results)

if __name__ == "__main__":
    df_results = get_top_5_sklearn_with_params_cc18()
    
    print("\n--- PRZYKŁADOWE WYNIKI ---")
    if not df_results.empty:

        top_model = df_results.iloc[0]
        print(f"Zadanie: {top_model['Dataset Name']}")
        print(f"Model: {top_model['Flow Name']}")
        print(f"AUC: {top_model['Score (AUC)']}")
        print("Parametry:")
        import json
        print(json.dumps(json.loads(top_model['Hyperparameters']), indent=2))
    
    # df_results.to_csv("openml_cc18_sklearn_top5_params.csv", index=False)

1. Pobieranie konfiguracji OpenML-CC18...
2. Znaleziono 35 zadań klasyfikacji binarnej.
3. Pobieranie modeli i ich hiperparametrów (może to potrwać)...

--- Zadanie: kr-vs-kp (ID: 3) ---
   [1/5] Pobrano parametry dla modelu (AUC: 0.9999)
   [2/5] Pobrano parametry dla modelu (AUC: 0.9999)
   [3/5] Pobrano parametry dla modelu (AUC: 0.9999)
   [4/5] Pobrano parametry dla modelu (AUC: 0.9999)
   [5/5] Pobrano parametry dla modelu (AUC: 0.9999)

--- Zadanie: breast-w (ID: 15) ---
   [1/5] Pobrano parametry dla modelu (AUC: 0.9953)
   [2/5] Pobrano parametry dla modelu (AUC: 0.9952)
   [3/5] Pobrano parametry dla modelu (AUC: 0.9952)
   [4/5] Pobrano parametry dla modelu (AUC: 0.9952)
   [5/5] Pobrano parametry dla modelu (AUC: 0.9952)

--- Zadanie: credit-approval (ID: 29) ---
   [1/5] Pobrano parametry dla modelu (AUC: 0.9386)
   [2/5] Pobrano parametry dla modelu (AUC: 0.9372)
   [3/5] Pobrano parametry dla modelu (AUC: 0.9372)
   [4/5] Pobrano parametry dla modelu (AUC: 0.9371)
   [5/